In [1]:
import pandas as pd
import ast

In [2]:
!which python

/home/vscode/venv/bin/python


In [5]:
import kaggle
from kaggle.api.kaggle_api_extended import KaggleApi
api = KaggleApi()
api.authenticate()

In [ ]:
# Download the dataset
# api.dataset_download_files('shuyangli94/food-com-recipes-and-user-interactions', 'RAW_recipes.csv', unzip=True)

In [27]:
data = pd.read_csv('RAW_recipes.csv')
data.head(2)

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6


In [28]:
# Select only 'name' and 'steps' columns 
data = data[['name', 'steps', 'description', 'ingredients']]
# shape 
data.shape

In [29]:
# Count number of rows that contain 'please ignore' or 'none' string 
print(data[data['steps'].str.contains('please ignore')].shape)
print(data[data['steps'].str.contains('none')].shape)

(1, 4)
(103, 4)


In [30]:
# Remove rows that contain 'please ignore' or 'none' string
data = data[~data['steps'].str.contains('please ignore')]
data = data[~data['steps'].str.contains('none')]
data.shape

(231533, 4)

In [31]:
def clean_steps(data):
    # Convert the string representation of a list to an actual list
    data_list = ast.literal_eval(data)
    # Join the list elements into a single string
    return ', '.join(data_list)

data['steps'] = data['steps'].apply(clean_steps)
data['ingredients'] = data['ingredients'].apply(clean_steps)
data.head(2)

,name,steps,description,ingredients
0,arriba baked winter squash mexican style,"make a choice and proceed with recipe, dependi...",autumn is my favorite time of year to cook! th...,"winter squash, mexican seasoning, mixed spice,..."
1,a bit different breakfast pizza,"preheat oven to 425 degrees f, press dough int...",this recipe calls for the crust to be prebaked...,"prepared pizza crust, sausage patty, eggs, mil..."


In [32]:
# assemble lines: concatenate 'steps' with 'name'
lines = data.apply(lambda row: str(row['name']) + 
                   '; ' + 
                   str(row['steps']).replace("\n", ' ') + 
                   '; ' + 
                   str(row['description']) + 
                   '; ' + 
                   str(row['ingredients']), axis=1).tolist()
sorted(lines, key=len)[:10]


['cherry coke; blend and chill then serve; a friend; coke, kool-aid',
 'chicken test; cook it, eat it; i like chicken; banana, apple, milk',
 'delicious water; pour the water, delicious; water; water, lemon wedge',
 'twin peaks martini; shake and strain; nan; vodka, whiskey, triple sec',
 'deluxe scrambled eggs; cook in a pan; good eggs; egg, bacon, tomatoes, spinach',
 'tahini substitution; mix well; good to know.; tahini, peanut butter, sesame oil',
 'cape cod; shake with ice and serve on the rocks; nan; vodka, lime, cranberry juice',
 "polar blast; mix, enjoy; great on a winter's night.; brandy, rum, vanilla ice cream",
 'homemade sour cream with yogurt; mix and use; easy peasy!; plain yogurt, lemon juice',
 'maple milk; combine in a glass; very easy and tasty drink to make.; milk, maple syrup']

In [33]:
data.head(3)

,name,steps,description,ingredients
0,arriba baked winter squash mexican style,"make a choice and proceed with recipe, dependi...",autumn is my favorite time of year to cook! th...,"winter squash, mexican seasoning, mixed spice,..."
1,a bit different breakfast pizza,"preheat oven to 425 degrees f, press dough int...",this recipe calls for the crust to be prebaked...,"prepared pizza crust, sausage patty, eggs, mil..."
2,all in the kitchen chili,"brown ground beef in large pot, add chopped on...",this modified version of 'mom's' chili was a h...,"ground beef, yellow onions, diced tomatoes, to..."
